# ML: Assignment 3

## Imports

In [1]:
from src.data_loading import DataLoader
from src.data_processing import Process

from src.next_word_model import NextWordModel

from src.prediction_evaluation import Evaluator

## Pipeline

### Data Loading & Processing

In [2]:
dl = DataLoader()
process = Process(dl)

In [3]:
X_train, X_test, y_train, y_test = process.process(force=False)


[Errno 2] No such file or directory: '/Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/_objects/data.pkl'
No preprocessed file to load - <force> will be ignored.


/Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/data_processing.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["hasLink"] = df.text.str.contains(url_regex)
/Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/data_processing.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["hasLink"] = df.text.str.contains(url_regex)


In [4]:
print(X_train)

{'train': {'X': array([1457,    1,  359, ...,   47,    6,  368]), 'y': array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)}, 'test': {'X': array([322,   4, 899, ...,  67,   6,  24]), 'y': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)}}


### Model Init & Training

In [5]:
model = NextWordModel(
    sequence_length = process.sequence_length,
    vocab_size = process.vocab_size

)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             52640     
                                                                 
 lstm (LSTM)                 (None, 1, 100)            44400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 5264)              531664    
                                                                 
Total params: 719,204
Trainable params: 719,204
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model.train(X_train, y_train)

NameError: name 'X_train' is not defined

### Model Evaluation

In [ ]:
ev = Evaluator(model)

In [ ]:
ev.evaluate_lemma(X_test, y_test)